In [6]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [7]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [8]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    return chat(messages, stop_sequences=["```"])

In [11]:
dataset = generate_dataset()

with open("eval_dataset.json", "w") as f:
    json.dump(json.loads(dataset), f, indent=2)

In [ ]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
    Please solve the following task:
    {test_case['task']}
    """
    messages = []
    add_user_message(messages, prompt)
    return chat(messages)

def run_test_case(test_case):
    """Calls run_prompt, then grade the result"""
    output = run_prompt(test_case)
    # TODO - Grade the output
    score = 10

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
    }

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each test case"""
    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    return results

In [15]:
with open("eval_dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

[{'output': '# AWS S3 Region Extraction Function\n\nHere\'s a complete solution with multiple approaches:\n\n## Basic Implementation\n\n```python\nimport boto3\nfrom botocore.exceptions import ClientError\n\ndef get_region_from_s3_uri(s3_uri):\n    """\n    Extract AWS region from an S3 bucket URI.\n    \n    Args:\n        s3_uri (str): S3 URI in format \'s3://bucket-name/key\' or \'s3://bucket-name\'\n        \n    Returns:\n        str: AWS region name (e.g., \'us-east-1\')\n        \n    Raises:\n        ValueError: If URI format is invalid\n        ClientError: If bucket doesn\'t exist or access is denied\n    """\n    # Parse the S3 URI\n    if not s3_uri.startswith(\'s3://\'):\n        raise ValueError("Invalid S3 URI format. Must start with \'s3://\'")\n    \n    # Extract bucket name\n    parts = s3_uri[5:].split(\'/\')\n    bucket_name = parts[0]\n    \n    if not bucket_name:\n        raise ValueError("Invalid S3 URI format. No bucket name provided")\n    \n    # Create S3 c